<a href="https://colab.research.google.com/github/avinashkarkada/GROMACS-COLAB/blob/main/APO_MD_GROMACS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Download and unzip the compressed folder of GROMACS 2020.6 version
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/software/gromacs.tar.gz
!tar xzf gromacs.tar.gz

In [ ]:
!pip install cmake --upgrade

In [ ]:
%%bash
source /content/gromacs/bin/GMXRC
pwd
gmx -h

In [ ]:
import os
from pathlib import Path 

#Then, we define the path of the folder we want to create.
#Notice that the HOME folder for a hosted runtime in colab is /content/
mdpath = Path("/content/I1171T/")

#Now, we create the folder using the os.mkdir() command
#The if conditional is just to check whether the folder already exists
#In which case, python returns an error
if os.path.exists(mdpath):
  print("path already exists")
if not os.path.exists(mdpath):
  os.mkdir(mdpath)
  print("path was succesfully created")

os.chdir(mdpath)
!pwd

In [ ]:
from google.colab import drive
import os
from pathlib import Path

drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

Google_Drive_Path = '/content/drive/MyDrive/I1171T' #@param {type:"string"}
workDir = Google_Drive_Path


In [ ]:
!cp $workDir/* $mdpath
!cp -r $workDir/*.ff $mdpath


cp: -r not specified; omitting directory '/content/drive/MyDrive/I1171T/gromos54a7.ff'


In [ ]:
%%bash
source /content/gromacs/bin/GMXRC
echo "1" | gmx pdb2gmx -f protein.pdb -o protein.gro -water spce -ignh 



In [ ]:
%%bash
source /content/gromacs/bin/GMXRC

gmx editconf -f protein.gro -o newbox.gro -c -d 0.5 -bt cubic
gmx grompp -f vacuum.mdp -c newbox.gro -p topol.top -o protein-EM-vacuum.tpr -maxwarn 2
gmx mdrun -v -deffnm protein-EM-vacuum -c protein-EM-vacuum.pdb
gmx mdrun -v -deffnm protein-EM-vacuum
gmx solvate -cp protein-EM-vacuum.gro -cs spc216.gro -p topol.top -o solv.gro
gmx grompp -f em.mdp -c solv.gro -p topol.top -o ions.tpr -maxwarn 2
gmx genion -s ions.tpr -o solv_ions.gro -p topol.top -conc 0.15  -neutral -pname NA -nname CL <<< 13
gmx grompp -f em.mdp -c solv_ions.gro -p topol.top -o em.tpr -maxwarn 2
gmx mdrun -v -deffnm em
gmx grompp -f nvt.mdp -c em.gro -p topol.top -o nvt.tpr -r em.gro -maxwarn 2
gmx mdrun -deffnm nvt -v
gmx grompp -f npt.mdp -c nvt.gro -t nvt.cpt -p topol.top -o npt-1000.tpr -r em.gro -maxwarn 2
gmx mdrun -deffnm npt-1000  -v

In [ ]:
%%bash
source /content/gromacs/bin/GMXRC



arr=(*_*_*_*.itp) && echo "${arr[0]}"

filename=${arr[0]}

if [ -f "${filename}" ]

then

vim -c ":%s/1000  1000  1000/100     100     100/g" -c wq <<< E ${arr[0]}
vim -c ":%s/1000  1000  1000/100     100     100/g" -c wq <<< E ${arr[1]}
gmx grompp -f npt.mdp -c npt-1000.gro -t npt-1000.cpt -p topol.top -o npt-100.tpr -r em.gro -maxwarn 2
gmx mdrun -deffnm npt-100  -v
vim -c ":%s/100     100     100/10     10     10/g" -c wq <<< E ${arr[0]}
vim -c ":%s/100     100     100/10     10     10/g" -c wq <<< E ${arr[1]}
gmx grompp -f npt.mdp -c npt-100.gro -t npt-100.cpt -p topol.top -o npt-10.tpr -r em.gro -maxwarn 2
gmx mdrun -deffnm npt-10  -v
vim -c ":%s/10     10     10/1     1     1/g" -c wq <<< E ${arr[0]}
vim -c ":%s/10     10     10/1     1     1/g" -c wq <<< E ${arr[1]}

else

vim -c ":%s/1000  1000  1000/100     100     100/g" -c wq <<< E posre.itp
gmx grompp -f npt.mdp -c npt-1000.gro -t npt-1000.cpt -p topol.top -o npt-100.tpr -r em.gro -maxwarn 2
gmx mdrun -deffnm npt-100  -v
vim -c ":%s/100     100     100/10     10     10/g" -c wq <<< E posre.itp
gmx grompp -f npt.mdp -c npt-100.gro -t npt-100.cpt -p topol.top -o npt-10.tpr -r em.gro -maxwarn 2
gmx mdrun -deffnm npt-10  -v
vim -c ":%s/10     10     10/1     1     1/g" -c wq <<< E posre.itp
fi

gmx grompp -f npt.mdp -c npt-10.gro -t npt-10.cpt -p topol.top -o npt-1.tpr -r em.gro -maxwarn 2
gmx mdrun -deffnm npt-1 -v
vim -c ":%s/define/;define/g" -c wq <<< E npt.mdp
gmx grompp -f npt.mdp -c npt-1.gro -t npt-1.cpt -p topol.top -o npt.tpr -r em.gro -maxwarn 2
gmx mdrun -deffnm npt -v
gmx grompp -f md.mdp -c npt.gro -t npt.cpt -p topol.top -o protein-10ns.tpr -maxwarn 2


In [ ]:
#Production run
%%bash
source /content/gromacs/bin/GMXRC
gmx mdrun -deffnm protein-10ns -v


In [ ]:
#Analysis
%%bash
source /content/gromacs/bin/GMXRC
printf '1\n1' | gmx trjconv -s $pid-1ns.tpr -f $pid-1ns.xtc -o $pid-1ns.pdb -pbc cluster
printf '1\n1' | gmx trjconv -s $pid-1ns.tpr -f $pid-1ns.xtc -o $pid-0-1ns.xtc -pbc cluster
printf '4\n4' | gmx rms -s $pid-1ns.tpr -f $pid-1ns.pdb -o $pid-0-1-RMSD.xvg 
gmx rmsf -s $pid-1ns.tpr -f $pid-1ns.pdb -o $pid-0-1ns-RMSF.xvg -res <<< 3